# Game RecSys Model Building/Evaluation
Notebook for building and evaluating ML models of the game recommendation systems

## Load and format data

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set()

### Load user-game ratings data
#### Training data

In [2]:
X_train = pd.read_csv("train_test_split/X_train_3k.csv")
X_train = X_train.set_index("Unnamed: 0")
y_train = pd.read_csv("train_test_split/y_train_3k.csv")
y_train = y_train.set_index("Unnamed: 0")
# join and reset index
train_df = pd.merge(X_train, y_train, left_index=True, right_index=True, validate="1:1")
train_df = train_df.reset_index(drop=True)[["user_id", "item_id", "recommend"]].copy()

In [3]:
train_df.head()

,user_id,item_id,recommend
0,Drewmatic,8930,1
1,76561198080148447,377160,1
2,AleksoSmeksoHere,342380,1
3,gaboqse,108800,0
4,piedude,215470,1


#### Test data

In [4]:
X_test = pd.read_csv("train_test_split/X_test_3k.csv")
X_test = X_test.set_index("Unnamed: 0")
y_test = pd.read_csv("train_test_split/y_test_3k.csv")
y_test = y_test.set_index("Unnamed: 0")
# join and reset index
test_df = pd.merge(X_test, y_test, left_index=True, right_index=True, validate="1:1")
test_df = test_df.reset_index(drop=True)[["user_id", "item_id", "recommend"]].copy()

In [5]:
test_df.head()

,user_id,item_id,recommend
0,sickbubblez,386360,1
1,GetALifeStopLookingAtMyUrl,4000,1
2,kineticvine,1250,1
3,LeoNoHomo,200210,1
4,itsdandytime,4000,1


### Load game metadata

In [6]:
game_meta = pd.read_csv("train_test_split/processed_metadata.csv").rename(
    columns={"Unnamed: 0":"item_id"}).set_index("item_id")
game_meta.head()

,early_access,metascore,price,sentiment,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,Design &amp; Illustration,...,Stats,Steam Achievements,Steam Cloud,Steam Leaderboards,Steam Trading Cards,Steam Turn Notifications,Steam Workshop,SteamVR Collectibles,Tracked Motion Controllers,Valve Anti-Cheat enabled
item_id,,,,,,,,,,,,,,,,,,,,,
10,0,88.0,9.99,3,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
20,0,NaN,4.99,2,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
30,0,79.0,4.99,2,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
50,0,NaN,4.99,2,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
60,0,NaN,4.99,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


## Content-based filtering (CBF)

### Compute game-game similarity matrix

In [7]:
from sklearn.preprocessing import MinMaxScaler

In [8]:
# fill NaN:s with column mean values
game_meta_cbf = game_meta.fillna(game_meta.mean())

# normalize values in the metascore, price, and sentiment columns to a range of 0 - 1
scaler = MinMaxScaler()
scaled_cbf = scaler.fit_transform(game_meta_cbf[["metascore", "price", "sentiment"]])
game_meta_cbf[["metascore", "price", "sentiment"]] = scaled_cbf
game_meta_cbf.head()

,early_access,metascore,price,sentiment,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,Design &amp; Illustration,...,Stats,Steam Achievements,Steam Cloud,Steam Leaderboards,Steam Trading Cards,Steam Turn Notifications,Steam Workshop,SteamVR Collectibles,Tracked Motion Controllers,Valve Anti-Cheat enabled
item_id,,,,,,,,,,,,,,,,,,,,,
10,0,0.888889,0.012318,1.000000,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
20,0,0.735917,0.005835,0.833333,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
30,0,0.763889,0.005835,0.833333,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
50,0,0.735917,0.005835,0.833333,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
60,0,0.735917,0.005835,0.666667,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [9]:
# create the similarity matrix using the Pearson correlation coefficient
game_similarity_matrix = game_meta_cbf.T.corr(method="pearson")
game_similarity_matrix.head()

item_id,10,20,30,50,60,70,80,130,220,240,...,461560,462930,464780,466910,480631,485380,485890,495890,498240,512540
item_id,,,,,,,,,,,,,,,,,,,,,
10,1.000000,0.995876,0.996523,0.886343,0.987870,0.903771,0.874333,0.595771,0.345255,0.666608,...,0.168448,0.057541,0.271566,0.398367,0.084417,0.271758,0.169150,0.221772,0.200073,0.403472
20,0.995876,1.000000,0.999913,0.890223,0.996834,0.897767,0.882236,0.567854,0.320196,0.664101,...,0.136324,0.035855,0.230239,0.376656,0.062280,0.230463,0.138871,0.184451,0.166047,0.409524
30,0.996523,0.999913,1.000000,0.890122,0.996925,0.898915,0.881176,0.570038,0.322738,0.664492,...,0.139112,0.038510,0.233858,0.378615,0.065606,0.234082,0.142606,0.187232,0.169551,0.409397
50,0.886343,0.890223,0.890122,1.000000,0.887474,0.992939,0.997260,0.768895,0.423638,0.576950,...,0.284290,0.167338,0.424495,0.321012,0.224906,0.424705,0.327764,0.337321,0.348091,0.466629
60,0.987870,0.996834,0.996925,0.887474,1.000000,0.891217,0.878529,0.545535,0.304708,0.658711,...,0.115835,0.026715,0.203760,0.361192,0.056281,0.203988,0.125749,0.157876,0.147367,0.412284


### CBF algorithm
To calculate the rating score for a particulair user-game pair, this algorithm does the following:
1. obtain the list of other game id:s this user has rated
2. obtain the similarity scores of those games to the game of interest
3. calculate the rating for the game of interest using the following equation: $$r_{ik}=\frac{\sum_{j\neq k}{r_{ij}s_{jk}}}{\sum_{j\neq k}{s_{jk}}}$$ where $r_{ik}$, $r_{ij}$, and $s_{jk}$ are the desired rating of user **i** for game **k**, rating of game **j** by user **i**, and the similarity of game **j** with game **k** respectively.
4. If no similarities are available for calculating $r_{ik}$, the algorithm just returns the average rating of game **k** by all users

In [38]:
# CBF algorithm
def cbf_rate(user, item, data, similarity):
    """calculates the predicted rating for a game
    by a user
    data - training df
    similarity - game similarity matrix
    avg_ratings - average game ratings"""
    # compute average ratings in the training set
    r_avg = np.average(data[data["item_id"] == item]["recommend"].values)
    #print("average rating: %.2f" % (r_avg))
    # obtain similarity scores for the game
    try:
        game_s = similarity[item].to_frame()
    except KeyError:
        # no similarity data available, return average rating
        #print("No similarity data available, using average rating...")
        return r_avg
    # obtain ratings by this user for all items except the item of interest
    game_r = data[(data["user_id"] == user) & 
                  (data["item_id"] != item)][["item_id", "recommend"]].copy().set_index("item_id")
    #print("Available ratings data:")
    #print(game_r)
    if len(game_r) > 0:
        # ratings data available, compute rating
        game_r_s = game_r.merge(game_s, left_index=True, right_index=True)
        if len(game_r_s) > 0:
            # there is available similarity data
            ratings, sims = game_r_s.iloc[:,0].values, game_r_s.iloc[:,1].values
            #print(ratings, sims)
            r = np.dot(ratings, sims) / np.sum(sims)
            #print(r)
            return r
        else:
            # no available similarity data, return average rating
            return r_avg
    else:
        # ratings data not available, return average rating
        return r_avg
    
def cbf_predict(data, training_data, similarity):
    """predicts ratings for all user-item pairs in the
    passed dataset - data
    similarity - game similarity matrix"""
    user_item = data[["user_id", "item_id"]]
    with tqdm(total=len(user_item)) as pbar:
        for idx, row in user_item.iterrows():
            r = cbf_rate(row["user_id"], row["item_id"], training_data, similarity)
            user_item.loc[idx, "r_pred"] = r
            pbar.update(1)
    return user_item
        

### CBF evaluation
Evaluate the CBF algorithm on both the training and test sets and compare with the baseline models

In [19]:
from sklearn.metrics import mean_squared_error

In [39]:
# make predictions and compute MSE on the training set
train_cbf_pred = cbf_predict(train_df, train_df, game_similarity_matrix)
train_cbf_mse = mean_squared_error(train_df["recommend"], train_cbf_pred["r_pred"])
print("MSE: %.4f" %(train_cbf_mse))


MSE: 0.1284


In [40]:
# make predictions and compute MSE on the test set
test_cbf_pred = cbf_predict(test_df, train_df, game_similarity_matrix)
test_cbf_mse = mean_squared_error(test_df["recommend"], test_cbf_pred["r_pred"])
print("MSE: %.4f" %(test_cbf_mse))


MSE: 0.1181


Not better than the baseline model!

## Collaborative filtering (CF) using deep learning 

### Assemble the full game metadata set
Include also the game ids without any metadata available. For these algorithms we cannot have missing values

In [8]:
game_ids = pd.Series(train_df["item_id"].unique(), name="item_id").to_frame().set_index("item_id")
full_meta = pd.merge(game_ids, game_meta, how="left", left_index=True, right_index=True)
full_meta.shape

(1346, 62)

#### Fill NaN:s and normalize values in `metascore`, `price`, and `sentiment` columns

In [9]:
# fill NaN:s with column mean values
full_meta.fillna(full_meta.mean(), inplace=True)

# normalize values in the 2 columns
scaler = MinMaxScaler()
scaled = scaler.fit_transform(full_meta[["metascore", "price", "sentiment"]])
full_meta[["metascore", "price", "sentiment"]] = scaled

full_meta.head()

,early_access,metascore,price,sentiment,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,Design &amp; Illustration,...,Stats,Steam Achievements,Steam Cloud,Steam Leaderboards,Steam Trading Cards,Steam Turn Notifications,Steam Workshop,SteamVR Collectibles,Tracked Motion Controllers,Valve Anti-Cheat enabled
item_id,,,,,,,,,,,,,,,,,,,,,
8930,0.000000,0.916667,0.038251,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,1.000000,0.000000,1.00000,0.000000,0.000000,0.00000,0.000000,0.000000
377160,0.000000,0.833333,0.038251,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,1.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000
342380,0.050633,0.735917,0.021977,0.764838,0.594991,0.303972,0.004318,0.000864,0.124352,0.001727,...,0.119898,0.637755,0.417517,0.213435,0.52551,0.002551,0.142857,0.00085,0.005952,0.064626
108800,0.000000,0.735917,0.038251,0.833333,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000
215470,0.000000,0.597222,0.009725,0.666667,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,1.00000,0.000000,0.000000,0.00000,0.000000,1.000000


### Recreate user-item indexes (needed for embedding layers)

In [10]:
# create new indexes for unique users/items
unique_users = pd.Series(train_df["user_id"].unique(),
                         name="user_name").to_frame().reset_index().rename(columns={"index":"user_idx"})

unique_items = pd.Series(train_df["item_id"].unique(),
                         name="original_id").to_frame().reset_index().rename(columns={"index":"item_idx"})


In [11]:
# join back on training/test sets
def assemble_reindexed_dataset(df):
    # join back on training/test sets
    df_user_idx = pd.merge(df, unique_users, how="left",
                              left_on="user_id",
                              right_on="user_name").drop(columns="user_name")
    df_idx = pd.merge(df_user_idx, unique_items, how="left",
                        left_on="item_id",
                        right_on="original_id").drop(columns="original_id")
    # join with game metadata
    df_full_idx = pd.merge(df_idx, full_meta, how="left",
                              left_on="item_id",
                              right_on="item_id").iloc[:,2:]
    return df_full_idx


In [12]:
train_full_idx = assemble_reindexed_dataset(train_df)
test_full_idx = assemble_reindexed_dataset(test_df)

In [13]:
train_full_idx.head()

,recommend,user_idx,item_idx,early_access,metascore,price,sentiment,Action,Adventure,Animation &amp; Modeling,...,Stats,Steam Achievements,Steam Cloud,Steam Leaderboards,Steam Trading Cards,Steam Turn Notifications,Steam Workshop,SteamVR Collectibles,Tracked Motion Controllers,Valve Anti-Cheat enabled
0,1,0,0,0.000000,0.916667,0.038251,1.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,1.000000,0.000000,1.00000,0.000000,0.000000,0.00000,0.000000,0.000000
1,1,1,1,0.000000,0.833333,0.038251,0.500000,0.000000,0.000000,0.000000,...,0.000000,1.000000,1.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000
2,1,2,2,0.050633,0.735917,0.021977,0.764838,0.594991,0.303972,0.004318,...,0.119898,0.637755,0.417517,0.213435,0.52551,0.002551,0.142857,0.00085,0.005952,0.064626
3,0,3,3,0.000000,0.735917,0.038251,0.833333,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000
4,1,4,4,0.000000,0.597222,0.009725,0.666667,1.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,0.000000,1.00000,0.000000,0.000000,0.00000,0.000000,1.000000


### Basic user-item embedding model

In [72]:
from keras.models import Model
from keras.layers import Input, Embedding, Dense, Flatten, Activation, BatchNormalization
from keras.layers import concatenate, dot
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import keras.backend as K

In [101]:
# define model variables
n_users = len(unique_users)
n_items = len(unique_items)
embedding_size_users = 16
embedding_size_items = 16

# create user path
user_input = Input(shape=(1,), name="user")
user_embeddings = Embedding(n_users, 
                            embedding_size_users, 
                            embeddings_regularizer=l2(0.005),
                            embeddings_initializer="glorot_normal",
                            name="user_embeddings")(user_input)
user_flat = Flatten(name="user_flattened")(user_embeddings)

# create item path
item_input = Input(shape=(1,), name="item")
item_embeddings = Embedding(n_items, 
                            embedding_size_items,
                            embeddings_regularizer=l2(0.005),
                            embeddings_initializer="glorot_normal",
                            name="item_embeddings")(item_input)
item_flat = Flatten(name="item_flattened")(item_embeddings)

# combine the two paths
prod = dot([user_flat, item_flat], 1, name="predicted_ratings")
# normalize input and constrain outputs to 0 - 1 range
prod_norm = BatchNormalization()(prod)
output = Activation("sigmoid")(prod_norm)

# compile the model
cf_model = Model(inputs=[user_input, item_input], outputs=output)
cf_model.compile("adam", loss="mse", metrics=["mse"])
cf_model.summary()

In [98]:
# get the data
X_train = train_full_idx.iloc[:, [1, 2]].values
y_train = train_full_idx.iloc[:, 0].values
X_test = test_full_idx.iloc[:, [1, 2]].values
y_test = test_full_idx.iloc[:, [0]].values

# format the training data
X_train_u = X_train[:,0].reshape(-1,1)
X_train_i = X_train[:,1].reshape(-1,1)
y_train = y_train.reshape(-1,1)
# format the test data
X_test_u = X_test[:,0].reshape(-1,1)
X_test_i = X_test[:,1].reshape(-1,1)
y_test = y_test.reshape(-1,1)

In [102]:
# summarize the model
cbf_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
item (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
user_embeddings (Embedding)     (None, 1, 16)        110496      user[0][0]                       
__________________________________________________________________________________________________
item_embeddings (Embedding)     (None, 1, 16)        21536       item[0][0]                       
__________________________________________________________________________________________________
user_flatt

In [103]:
# define callbacks
reduce_lr = ReduceLROnPlateau(patience=5, verbose=1)
early_stop = EarlyStopping(min_delta=0.0001, patience=10, verbose=1)

# train model for a few epochs
cbf_model.fit(x={"user":X_train_u, "item":X_train_i}, 
              y=y_train, 
              batch_size=32, 
              epochs=50, 
              verbose=2, 
              validation_split=0.2,
              callbacks=[reduce_lr, early_stop]
             )


Train on 19975 samples, validate on 4994 samples
Epoch 1/50
 - 5s - loss: 0.2103 - mean_squared_error: 0.2018 - val_loss: 0.1683 - val_mean_squared_error: 0.1683
Epoch 2/50
 - 3s - loss: 0.1443 - mean_squared_error: 0.1443 - val_loss: 0.1349 - val_mean_squared_error: 0.1349
Epoch 3/50
 - 3s - loss: 0.1206 - mean_squared_error: 0.1206 - val_loss: 0.1218 - val_mean_squared_error: 0.1218
Epoch 4/50
 - 3s - loss: 0.1109 - mean_squared_error: 0.1109 - val_loss: 0.1167 - val_mean_squared_error: 0.1167
Epoch 5/50
 - 3s - loss: 0.1069 - mean_squared_error: 0.1069 - val_loss: 0.1148 - val_mean_squared_error: 0.1148
Epoch 6/50
 - 3s - loss: 0.1051 - mean_squared_error: 0.1051 - val_loss: 0.1141 - val_mean_squared_error: 0.1141
Epoch 7/50
 - 3s - loss: 0.1044 - mean_squared_error: 0.1044 - val_loss: 0.1140 - val_mean_squared_error: 0.1140
Epoch 8/50
 - 3s - loss: 0.1041 - mean_squared_error: 0.1041 - val_loss: 0.1140 - val_mean_squared_error: 0.1140
Epoch 9/50
 - 3s - loss: 0.1040 - mean_squared_

In [112]:
# evaluate on the test set
cf_model.evaluate(x={"user":X_test_u, "item":X_test_i}, y=y_test)

8323/8323 [==============================] - ETA:  - ETA:  - ETA:  - 0s 18us/step


[0.10103787400938904, 0.10103787400938904]

### Deep learning of user-item preferences

In [131]:
from keras.layers import Dropout

In [175]:
# define model variables
n_users = len(unique_users)
n_items = len(unique_items)
embedding_size_users = 32
embedding_size_items = 32
l2_param = 0.001
dropout_rate = 0.5

# create user path
user_input = Input(shape=(1,), name="user")
user_embeddings = Embedding(n_users, 
                            embedding_size_users, 
                            embeddings_regularizer=l2(l2_param),
                            embeddings_initializer="glorot_normal",
                            name="user_embeddings")(user_input)
user_flat = Flatten(name="user_flattened")(user_embeddings)

# create item path
item_input = Input(shape=(1,), name="item")
item_embeddings = Embedding(n_items, 
                            embedding_size_items,
                            embeddings_regularizer=l2(l2_param),
                            embeddings_initializer="glorot_normal",
                            name="item_embeddings")(item_input)
item_flat = Flatten(name="item_flattened")(item_embeddings)

# combine the two paths
user_item = concatenate([user_flat, item_flat], name="user_item")
user_item = Dropout(dropout_rate)(user_item)

# hidden layer 1
interaction = Dense(32, kernel_regularizer=l2(l2_param), bias_regularizer=l2(l2_param))(user_item)
interaction = BatchNormalization()(interaction)
interaction = Activation("relu")(interaction)
interaction = Dropout(dropout_rate)(interaction)

# hidden layer 2
# interaction = Dense(32, kernel_regularizer=l2(l2_param), bias_regularizer=l2(l2_param))(interaction)
# interaction = BatchNormalization()(interaction)
# interaction = Activation("relu")(interaction)
# interaction = Dropout(dropout_rate)(interaction)

# output layer
pred = Dense(1, activation="sigmoid")(interaction)

# compile the model
dcf_model = Model(inputs=[user_input, item_input], outputs=pred)
dcf_model.compile("adam", loss="mse", metrics=["mse"])
dcf_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
item (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
user_embeddings (Embedding)     (None, 1, 32)        220992      user[0][0]                       
__________________________________________________________________________________________________
item_embeddings (Embedding)     (None, 1, 32)        43072       item[0][0]                       
__________________________________________________________________________________________________
user_flatt

In [176]:
# define callbacks
reduce_lr = ReduceLROnPlateau("val_mean_squared_error", patience=5, verbose=1)
early_stop = EarlyStopping("val_mean_squared_error", min_delta=0.0001, patience=5, verbose=1)

# train model for a few epochs
dcf_model.fit(x={"user":X_train_u, "item":X_train_i}, 
              y=y_train, 
              batch_size=32, 
              epochs=3, 
              verbose=2, 
              validation_split=0.2,
              callbacks=[reduce_lr, early_stop]
             )


Train on 19975 samples, validate on 4994 samples
Epoch 1/3
 - 9s - loss: 0.1653 - mean_squared_error: 0.1216 - val_loss: 0.1399 - val_mean_squared_error: 0.1045
Epoch 2/3
 - 4s - loss: 0.1321 - mean_squared_error: 0.0970 - val_loss: 0.1343 - val_mean_squared_error: 0.1007
Epoch 3/3
 - 4s - loss: 0.1230 - mean_squared_error: 0.0875 - val_loss: 0.1359 - val_mean_squared_error: 0.1017


In [177]:
# evaluate on the test set
dcf_model.evaluate(x={"user":X_test_u, "item":X_test_i}, y=y_test)

8323/8323 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 0s 49us/step


[0.12482817158475384, 0.09067008720566974]

## Hybrid methods

### Combine game metadata as extra features

In [182]:
# format game metadata
X_train_meta = train_full_idx.iloc[:,3:].values
X_test_meta = test_full_idx.iloc[:,3:].values

In [190]:
# define model variables
n_users = len(unique_users)
l2_param = 0.005
meta_features = X_train_meta.shape[1]
embedding_size_users = meta_features

# create user path
user_input = Input(shape=(1,), name="user")
user_embeddings = Embedding(n_users, 
                            embedding_size_users, 
                            embeddings_regularizer=l2(l2_param),
                            embeddings_initializer="glorot_normal",
                            name="user_embeddings")(user_input)
user_embeddings = Flatten(name="user_flattened")(user_embeddings)

# create item path
item_meta = Input(shape=(meta_features,), name="item_meta")

# combine the two paths
pred = dot([user_embeddings, item_meta], 1, name="pred_ratings")
pred = BatchNormalization()(pred)
pred = Activation("sigmoid")(pred)

# compile the model
mcf_model = Model(inputs=[user_input, item_meta], outputs=pred)
mcf_model.compile("adam", loss="mse", metrics=["mse"])
mcf_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
user_embeddings (Embedding)     (None, 1, 62)        428172      user[0][0]                       
__________________________________________________________________________________________________
user_flattened (Flatten)        (None, 62)           0           user_embeddings[0][0]            
__________________________________________________________________________________________________
item_meta (InputLayer)          (None, 62)           0                                            
__________________________________________________________________________________________________
pred_ratin

In [191]:
# define callbacks
reduce_lr = ReduceLROnPlateau("val_mean_squared_error", patience=5, verbose=1)
early_stop = EarlyStopping("val_mean_squared_error", min_delta=0.0001, patience=5, verbose=1)

# train model for a few epochs
mcf_model.fit(x={"user":X_train_u, "item_meta":X_train_meta}, 
              y=y_train, 
              batch_size=32, 
              epochs=20, 
              verbose=2, 
              validation_split=0.2,
              callbacks=[reduce_lr, early_stop]
             )


Train on 19975 samples, validate on 4994 samples
Epoch 1/20
 - 9s - loss: 0.2467 - mean_squared_error: 0.2118 - val_loss: 0.2032 - val_mean_squared_error: 0.1711
Epoch 2/20
 - 4s - loss: 0.1551 - mean_squared_error: 0.1276 - val_loss: 0.1698 - val_mean_squared_error: 0.1357
Epoch 3/20
 - 4s - loss: 0.1366 - mean_squared_error: 0.1035 - val_loss: 0.1602 - val_mean_squared_error: 0.1234
Epoch 4/20
 - 4s - loss: 0.1285 - mean_squared_error: 0.0925 - val_loss: 0.1587 - val_mean_squared_error: 0.1189
Epoch 5/20
 - 4s - loss: 0.1253 - mean_squared_error: 0.0880 - val_loss: 0.1596 - val_mean_squared_error: 0.1171
Epoch 6/20
 - 4s - loss: 0.1247 - mean_squared_error: 0.0855 - val_loss: 0.1609 - val_mean_squared_error: 0.1171
Epoch 7/20
 - 4s - loss: 0.1244 - mean_squared_error: 0.0843 - val_loss: 0.1611 - val_mean_squared_error: 0.1185
Epoch 8/20
 - 4s - loss: 0.1236 - mean_squared_error: 0.0830 - val_loss: 0.1605 - val_mean_squared_error: 0.1182
Epoch 9/20
 - 4s - loss: 0.1250 - mean_squared_

In [192]:
# evaluate on the test set
mcf_model.evaluate(x={"user":X_test_u, "item_meta":X_test_meta}, y=y_test)

8323/8323 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 0s 39us/step


[0.15593609781552004, 0.10980831845210863]

### Deep model with metadata features

In [197]:
# define model variables
n_users = len(unique_users)
n_items = len(unique_items)
embedding_size_users = 64
meta_features = X_train_meta.shape[1]

l2_param = 0.001
dropout_rate = 0.5

# create user path
user_input = Input(shape=(1,), name="user")
user_embeddings = Embedding(n_users, 
                            embedding_size_users, 
                            embeddings_regularizer=l2(l2_param),
                            embeddings_initializer="glorot_normal",
                            name="user_embeddings")(user_input)
user_embeddings = Flatten(name="user_flattened")(user_embeddings)

# create item path
item_meta = Input(shape=(meta_features,), name="item_meta")

# combine the two paths
user_item = concatenate([user_embeddings, item_meta], name="user_item")
user_item = Dropout(dropout_rate)(user_item)

# hidden layer 1
interaction = Dense(32, kernel_regularizer=l2(l2_param), bias_regularizer=l2(l2_param))(user_item)
interaction = BatchNormalization()(interaction)
interaction = Activation("relu")(interaction)
interaction = Dropout(dropout_rate)(interaction)

# hidden layer 2
# interaction = Dense(32, kernel_regularizer=l2(l2_param), bias_regularizer=l2(l2_param))(interaction)
# interaction = BatchNormalization()(interaction)
# interaction = Activation("relu")(interaction)
# interaction = Dropout(dropout_rate)(interaction)

# output layer
pred = Dense(1, activation="sigmoid")(interaction)

# compile the model
dmcf_model = Model(inputs=[user_input, item_meta], outputs=pred)
dmcf_model.compile("adam", loss="mse", metrics=["mse"])
dmcf_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
user (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
user_embeddings (Embedding)     (None, 1, 64)        441984      user[0][0]                       
__________________________________________________________________________________________________
user_flattened (Flatten)        (None, 64)           0           user_embeddings[0][0]            
__________________________________________________________________________________________________
item_meta (InputLayer)          (None, 62)           0                                            
__________________________________________________________________________________________________
user_item 

In [198]:
# define callbacks
reduce_lr = ReduceLROnPlateau("val_mean_squared_error", patience=5, verbose=1)
early_stop = EarlyStopping("val_mean_squared_error", min_delta=0.0001, patience=10, verbose=1)

# train model for a few epochs
dmcf_model.fit(x={"user":X_train_u, "item_meta":X_train_meta}, 
              y=y_train, 
              batch_size=32, 
              epochs=20, 
              verbose=2, 
              validation_split=0.2,
              callbacks=[reduce_lr, early_stop]
             )


Train on 19975 samples, validate on 4994 samples
Epoch 1/20
 - 10s - loss: 0.1600 - mean_squared_error: 0.1393 - val_loss: 0.1202 - val_mean_squared_error: 0.1104
Epoch 2/20
 - 5s - loss: 0.1174 - mean_squared_error: 0.1098 - val_loss: 0.1150 - val_mean_squared_error: 0.1092
Epoch 3/20
 - 5s - loss: 0.1120 - mean_squared_error: 0.1073 - val_loss: 0.1137 - val_mean_squared_error: 0.1099
Epoch 4/20
 - 5s - loss: 0.1087 - mean_squared_error: 0.1055 - val_loss: 0.1129 - val_mean_squared_error: 0.1101
Epoch 5/20
 - 5s - loss: 0.1075 - mean_squared_error: 0.1051 - val_loss: 0.1140 - val_mean_squared_error: 0.1118
Epoch 6/20
 - 5s - loss: 0.1060 - mean_squared_error: 0.1039 - val_loss: 0.1128 - val_mean_squared_error: 0.1109
Epoch 7/20
 - 5s - loss: 0.1055 - mean_squared_error: 0.1037 - val_loss: 0.1119 - val_mean_squared_error: 0.1102

Epoch 00007: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 8/20
 - 5s - loss: 0.1047 - mean_squared_error: 0.1031 - val_loss: 0.11